# 1. Libraries

In [7]:
import splusdata #Please certify that you have the latest version (> 3.83) of splusdata installed! 
from getpass import getpass
from splusdata.vacs import SQGClass
from astropy.table import Table
import pandas as pd

# 2. Read data

## Option 1: Read data from S-PLUS query

In [2]:
login = 'marixko' #INSERT YOUR S-PLUS LOGIN HERE (register: https://splus.cloud)
pswd = getpass() #this will ask for your password
conn = splusdata.connect(login, pswd)

You have access to internal data


In [ ]:
field = 'SPLUS-s29s34'

query = f"""select det.id, det.ra, det.dec, det.fwhm_n, det.a, det.b, det.kron_radius, u.u_iso,
						z.z_iso, g.g_iso, r.r_iso, i.i_iso,
						j0378.j0378_iso, j0395.j0395_iso, j0410.j0410_iso, 
						j0430.j0430_iso, j0515.j0515_iso,  j0660.j0660_iso, 
						j0861.j0861_iso from idr3.detection_image as det join
						idr3.g_band as g on g.id = det.id join
						idr3.z_band as z on z.id = det.id join
						idr3.r_band as r on r.id = det.id join
						idr3.i_band as i on i.id = det.id join
						idr3.u_band as u on u.id = det.id join
						idr3.j0378_band as j0378 on j0378.id = det.id join
						idr3.j0395_band as j0395 on j0395.id = det.id join
						idr3.j0410_band as j0410 on j0410.id = det.id join
						idr3.j0430_band as j0430 on j0430.id = det.id join
						idr3.j0515_band as j0515 on j0515.id = det.id join
						idr3.j0660_band as j0660 on j0660.id = det.id join
						idr3.j0861_band as j0861 on j0861.id = det.id 
						where det.field = '{field}'"""


In [ ]:
data = conn.query(query)

finished


In [ ]:
#Save data to csv:
#data.to_pandas().to_csv("SPLUS-s29s34.csv", sep=",", index=False)

## Option 2: Read data from folder

In [3]:
data = pd.read_table("SPLUS-s29s34.csv", sep=",")

# 3. Extinction correction (necessary step!)

In [4]:
from utils.extinction import correction
data = correction(data, save=False)

# 4. Classifying S-PLUS fields (Nakazono et al. 2021)

Before proceeding, please make sure that your data are previously corrected to extinction. Below we show three possible outputs by stipulating:

- model = "auto": the best classification will be returned (default and most recommended)
- model = "RF18": this classifier needs both S-PLUS and WISE features and the performance is usually better than RF16 (see Nakazono et al. 2021 for more details)
- model = "RF16": this classifier needs only S-PLUS features and can be used for objects without WISE counterpart


Initialize SQGClass as:

In [5]:
clf_auto = SQGClass(verbose=True)
clf_splus = SQGClass(model = "RF16", verbose=True)
clf_splus_wise = SQGClass(model = "RF18", verbose=True)

Loading both models...


/home/mariko/anaconda3/envs/research/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/mariko/anaconda3/envs/research/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


Loading model...
Loading model...


For clf_auto and clf_splus_wise, one must provide the W1 and W2 magnitudes from ALLWISE. We recommend that you set match_irsa = True, therefore it will automatically query from ALLWISE and cross-match with S-PLUS. 
To do so, please ensure that you run classify() one S-PLUS field per time, at maximum.

In [6]:
results = clf_auto.classify(data, match_irsa = True)
results

Querying ALLWISE catalogue via TapPlus(url='https://irsa.ipac.caltech.edu/TAP')...
Note that irsa_query() is not yet optimized for large data inputs. Please consider running classify() per S-PLUS field if irsa_query()=True.
Created TAP+ (v1.2.1) - Connection:
	Host: irsa.ipac.caltech.edu
	Use HTTPS: True
	Port: 443
	SSL Port: 443
INFO: Query finished. [astroquery.utils.tap.core]
Starting cross-match...


,CLASS,model_flag,PROB_QSO,PROB_STAR,PROB_GAL
0,0.0,1,0.540,0.20,0.260
1,0.0,1,0.585,0.19,0.225
2,2.0,1,0.450,0.09,0.460
3,0.0,1,0.460,0.19,0.350
4,0.0,1,0.505,0.27,0.225
...,...,...,...,...,...
62738,0.0,1,0.500,0.09,0.410
62739,2.0,0,0.050,0.15,0.800
62740,2.0,0,0.060,0.02,0.920
62741,2.0,1,0.240,0.10,0.660


In [8]:
results = clf_splus_wise.classify(data, match_irsa = True)
results

Querying ALLWISE catalogue via TapPlus(url='https://irsa.ipac.caltech.edu/TAP')...
Note that irsa_query() is not yet optimized for large data inputs. Please consider running classify() per S-PLUS field if irsa_query()=True.
Created TAP+ (v1.2.1) - Connection:
	Host: irsa.ipac.caltech.edu
	Use HTTPS: True
	Port: 443
	SSL Port: 443
INFO: Query finished. [astroquery.utils.tap.core]
Starting cross-match...


,CLASS,PROB_QSO,PROB_STAR,PROB_GAL
21,2,0.330,0.07,0.600
24,2,0.040,0.22,0.740
35,2,0.010,0.00,0.990
41,0,0.480,0.13,0.390
55,0,0.605,0.02,0.375
...,...,...,...,...
62733,0,0.600,0.06,0.340
62734,2,0.000,0.02,0.980
62737,1,0.000,1.00,0.000
62739,2,0.050,0.15,0.800


In [9]:
results = clf_splus.classify(data, match_irsa = True)
results

Parameter match_irsa is being set automatically to false.


,CLASS,PROB_QSO,PROB_STAR,PROB_GAL
0,0.0,0.540,0.20,0.260
1,0.0,0.585,0.19,0.225
2,2.0,0.450,0.09,0.460
3,0.0,0.460,0.19,0.350
4,0.0,0.505,0.27,0.225
...,...,...,...,...
62738,0.0,0.500,0.09,0.410
62739,2.0,0.020,0.03,0.950
62740,2.0,0.045,0.04,0.915
62741,2.0,0.240,0.10,0.660
